In [45]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import matplotlib
from datetime import datetime
import yfinance as yf
import crossStrategy
import qgrid

In [46]:
cerebro = bt.Cerebro()

data = bt.feeds.PandasData(dataname=yf.download('BTC-USD', '2022-01-01', '2023-01-01', auto_adjust=True)) 
cerebro.adddata(data)
 
strats = cerebro.optstrategy(
        crossStrategy.MaCrossStrategy,
        fast_length = range(1,20,2), 
        slow_length = range(25, 60, 5))
 
 
cerebro.broker.setcash(1000000.0)
 
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = "sharpe")
cerebro.addanalyzer(btanalyzers.DrawDown, _name = "drawdown")
cerebro.addanalyzer(btanalyzers.Returns, _name = "returns")
 
back = cerebro.run()

[*********************100%%**********************]  1 of 1 completed


In [47]:
par_list = [[x[0].params.fast_length, 
             x[0].params.slow_length,
             x[0].analyzers.returns.get_analysis()['rnorm100'], 
             x[0].analyzers.drawdown.get_analysis()['max']['drawdown'],
             x[0].analyzers.sharpe.get_analysis()['sharperatio']
            ] for x in back]

In [48]:
par_df = pd.DataFrame(par_list, columns = ['length_fast', 'length_slow', 'return', 'dd', 'sharpe'])

In [50]:
qgrid.enable()
qgrid.show_grid(par_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…